# Περιγραφή

Στο συγκεκριμένο notebook αποτυπώνεται η διαδικασία της επεξεργασία των δεδομένων (raw data), τα οποία βρίσκονται σε **xml** format, έτσι ώστε να αποτυπωθούν με τη μορφή ένος **DataFrame** (από τη βιβλιοθήκη pandas της python).

Η εν λόγω διαδικασία αποτελεί ένα παράδειγμα και αναφέρεται στα δεδομένα που εξήχθησαν από **έναν** άνθρωπο (subject). 

Η κατανόηση του παραδείγματος θα βοηθήσει στη γενίκευση της διαδικασίας για όλους τους χρήστες.

# Εισαγωγή βιβλιοθηκών

In [129]:
import os
import xml.etree.ElementTree as et
import pandas as pd

# Επεξεργασία δεδομένων

Προεπισκόπηση της μορφής των δεδομένων (xml format)

<img src=../Images/raw_data_preview_example.png align="left"/> 

### Πρώτη εκτίμηση για missing data


|  TAGS   |     CHILDREN      |
|---------|-------------------|
|  ID     |           -       |
| WRITING | DATE, TITLE, INFO |
| WRITING | DATE, TITLE, INFO |
| WRITING | DATE, TEXT, INFO  |
|   ...   |        ...        |
  
Παρατηρούμε ότι τα CHILDREN του WRITING στη **γενική περέπτωση** είναι τέσσερα. 

```python
{'DATE', 'TITLE', 'TEXT', 'INFO'}
```

Παρ' όλα αυτά -κατά περιπτώσεις- το πλήθος των CHILDREN διαφέρει. 
  
Έπειτα από σχετική εξέταση στα δεδομένα όλων των ανθρώπων (subjects), τα παραπανώ ισχύουν γενικά για όλα τα δεδομένα.

In [122]:
tree = et.parse('/home/kaml/Documents/ERPI/Data/data/subject1793.xml')
root = tree.getroot()

In [123]:
tag_names = {t.tag for t in root.findall('.//WRITING/*')}
print (tag_names)

{'DATE', 'TITLE', 'TEXT', 'INFO'}


In [124]:
for ID in root.findall('ID'):
    identity = ID.text
    print (identity)

subject1793


In [125]:
def check_if_NoneType(child):
    '''
    Avoid AttributeError: 'NoneType' object has no attribute 'text'
    '''
    if writing.find(child) is not None:
        return writing.find(child).text
    else:
        return 'NaN'

datetimes = []
texts     = []
infos     = []
titles    = []

for writing in root.findall('WRITING'):
    datetimes.append(check_if_NoneType('DATE'))
    texts.append(check_if_NoneType('TEXT'))
    infos.append(check_if_NoneType('INFO'))
    titles.append(check_if_NoneType('TITLE'))

In [126]:
zippedList =  list(zip(datetimes, titles, texts, infos))
df = pd.DataFrame(data=zippedList, columns=['DateTime','Title', 'Text', 'Info'])
df['DateTime'] = pd.to_datetime(df['DateTime'])
df['Subject'] = identity


In [127]:
df.head()

,DateTime,Title,Text,Info,Subject
0,2016-05-17 16:48:36,Worst experience with drugs you've ever had?,NaN,Reddit post,subject1793
1,2016-05-18 18:36:01,Dillion supply co,NaN,Reddit post,subject1793
2,2016-05-19 01:13:41,NaN,fuckin autocorrect lol,Reddit post,subject1793
3,2016-05-19 01:24:14,NaN,"didn't try to get inside, looked pretty hard t...",Reddit post,subject1793
4,2016-05-19 05:27:45,NaN,wondering the same,Reddit post,subject1793


In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573 entries, 0 to 572
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   DateTime  573 non-null    datetime64[ns]
 1   Title     573 non-null    object        
 2   Text      573 non-null    object        
 3   Info      573 non-null    object        
 4   Subject   573 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 22.5+ KB
